In [1]:
import pandas as pd
import geopandas as gpd
from datetime import date, datetime
import os
import re

pd.options.display.max_columns = None

In [36]:
fires = pd.DataFrame()
pathname = '../data'

for filename in os.listdir(pathname):
    if os.path.isfile(os.path.join(pathname, filename)) and 'Confirmed' in filename:
        if os.path.splitext(filename)[1] == '.csv':
            fires = pd.concat([fires, pd.read_csv(os.path.join(pathname, filename))], ignore_index=True)

fires = fires.drop('OBJECTID', axis=1).drop_duplicates()

In [37]:
def strip_time(entry):
    m = re.match('(\d{4}/\d{2}/\d{2})', entry)
    return m.group(1)

In [38]:
fires['confirmed_fire'] = True
fires['alarm_datetime'] = fires.apply(lambda row: strip_time(row.alm_date) + ' ' + row.alm_time, axis=1)
fires['clear_datetime'] = fires.apply(lambda row: strip_time(row.clr_date) + ' ' + row.clr_time, axis=1)
fires['alarm_datetime'] = pd.to_datetime(fires['alarm_datetime'])
fires['clear_datetime'] = pd.to_datetime(fires['clear_datetime'])

fires['alm_date'] = fires['alm_date'].apply(strip_time)
fires['clr_date'] = fires['clr_date'].apply(strip_time)

fires = fires.drop(['alm_time', 'clr_time', 'X', 'Y', 'LastUpdateDate', 'StartDate', 'EndDate'], axis=1)
fires = fires.sort_values('inci_no')



fires.head()

,FID,inci_no,descript,alm_date,station,number_,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,number,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,NaN,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,2716,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,NaN,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,NaN,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,2100,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,NaN,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,NaN,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,4222,True,2016-08-03 11:37:01,2016-08-03 14:39:18


In [39]:
years = sorted(fires['alarm_datetime'].dt.year.unique())
years

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [40]:
col_names = fires.columns
col_names = col_names.drop('number_', 'number').insert(5, 'number').drop_duplicates()
col_names
fires = fires[col_names]

fires.head()

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18


In [41]:
df = fires.loc[fires.alarm_datetime.dt.year == 2021]

In [42]:
import matplotlib.pyplot as plt 
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import io
from urllib.request import urlopen, Request
from PIL import Image


# This code uses a spoofing algorithm to avoid bounceback from OSM servers
#

def image_spoof(self, tile): # this function pretends not to be a Python script
    url = self._image_url(tile) # get the url of the street map API
    req = Request(url) # start request
    req.add_header('User-agent','Anaconda 3') # add user agent to request
    fh = urlopen(req) 
    im_data = io.BytesIO(fh.read()) # get image
    fh.close() # close url
    img = Image.open(im_data) # open image with PIL
    img = img.convert(self.desired_tile_form) # set image format
    return img, self.tileextent(tile), 'lower' # reformat for cartopy

In [43]:
def plot_fire_map(date_min, date_max):
    df = fires.loc[fires.alarm_datetime.dt.year == year]


    max_lat, min_lat = df.latitude.max(), df.latitude.min()
    max_lon, min_lon = df.longitude.max(), df.longitude.min()


    cimgt.OSM.get_image = image_spoof # reformat web request for street map spoofing
    osm_img = cimgt.OSM() # spoofed, downloaded street map


    fig = plt.figure(figsize=(12,15)) # open matplotlib figure
    ax = fig.add_subplot(1,1,1, projection=osm_img.crs) # project using coordinate reference system (CRS) of street map
    ax.set_global()
    zoom = 0.075 # for zooming out of center point
    extent = [min_lon, max_lon, min_lat, max_lat]
    ax.set_extent(extent) # set extents

    scale = np.ceil(-np.sqrt(2)*np.log(np.divide(zoom,350.0))) # empirical solve for scale based on zoom
    scale = (scale<20) and scale or 19 # scale cannot be larger than 19
    ax.add_image(osm_img, int(scale), zorder=0) # add OSM with zoom specification
    # NOTE: zoom specifications should be selected based on extent:
    # -- 2     = coarse image, select for worldwide or continental scales
    # -- 4-6   = medium coarseness, select for countries and larger states
    # -- 6-10  = medium fineness, select for smaller states, regions, and cities
    # -- 10-12 = fine image, select for city boundaries and zip codes
    # -- 14+   = extremely fine image, select for roads, blocks, buildings

    ax.scatter(df.longitude, df.latitude, alpha= 0.5, c='r', s=50,
                transform=ccrs.PlateCarree()) # NEED transform=crs.PlateCarree()

    plt.title('Minneapolis Fire Calls', size=24)
    plt.show() # show the plot

In [44]:
# plot_fire_map(2017)

In [45]:
# Years with some incorrect locations: 2016, 2020
fires.loc[fires.longitude > -90]

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38296,NaN,0030463,Passenger vehicle fire ...,2016/08/28,16,1515,Washburn,AVE,N,,,,,,,0.0,0.0,131,2016/08/29,1,1,True,2016-08-28 22:53:05,2016-08-29 00:42:15
38297,NaN,0030615,Passenger vehicle fire ...,2016/08/30,06,1300-130,Interstate 94 Wb,,,,,,,,,0.0,0.0,131,2016/08/30,1,1,True,2016-08-30 01:08:49,2016-08-30 01:44:16
38298,NaN,0030851,"Outside rubbish, trash or waste fire ...",2016/08/31,05,3129,Oakland,AVE,,,,,,,,0.0,0.0,151,2016/08/31,1,1,True,2016-08-31 20:10:30,2016-08-31 20:18:47
38299,NaN,0030887,Passenger vehicle fire ...,2016/09/01,06,,14,ST,W,,,,Willow,ST,,0.0,0.0,131,2016/09/01,1,1,True,2016-09-01 06:24:02,2016-09-01 07:13:31


In [3]:
import requests
import urllib.parse

address = 'Shivaji Nagar, Bangalore, KA 560001'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

12.9801559
77.6023714


In [47]:
fires.loc[fires.latitude == 0].head()

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18


In [48]:
test_row = fires.loc[fires.latitude == 0].iloc[2]
address = re.sub('\s+', ' ', f'{test_row.number} {test_row.street} {test_row.st_type} {test_row.st_suffix.strip()}, Minneapolis, MN')
address

'2100 Elliot AVE S, Minneapolis, MN'

In [49]:
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

44.9617
-93.261202


In [4]:
def get_lat(row):
    if row.latitude == 0:
        address = re.sub('\s+', ' ', f'{row.number}+{row.street}+{row.st_type}+{row.st_suffix.strip()}+Minneapolis+MN')
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

        response = requests.get(url).json()
        try:
            return response[0]["lat"]
        except:
            return np.nan
    else:
        return row.latitude

def get_lon(row):
    if row.longitude == 0:
        address = re.sub('\s+', ' ', f'{row.number}+{row.street}+{row.st_type}+{row.st_suffix.strip()}+Minneapolis+MN')
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

        response = requests.get(url).json()
        try:
            return response[0]["lon"]
        except:
            return np.nan
    else:
        return row.longitude

In [5]:
fires['new_lat'] = fires.apply(get_lat, axis=1)
fires['new_lon'] = fires.apply(get_lon, axis=1)

NameError: name 'fires' is not defined

In [53]:
print(len(fires.loc[fires.latitude != fires.new_lat]))
print(len(fires.loc[fires.longitude != fires.new_lon]))

64
64


In [54]:
fires.head()

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime,new_lat,new_lon
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14,NaN,NaN
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03,44.9627504,-93.3054853
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13,44.9617,-93.261202
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09,44.891355,-93.2468121
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18,NaN,NaN


In [55]:
fires.loc[fires.new_lat.isna()]

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime,new_lat,new_lon
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14,NaN,NaN
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18,NaN,NaN
38244,NaN,0027398,"Natural vegetation fire, Other ...",2016/08/05,05,,24,AVE,S,,,E,32,ST,,0.0,0.0,140,2016/08/05,1,1,True,2016-08-05 18:35:03,2016-08-05 18:44:37,NaN,NaN
38248,NaN,0027619,Brush or brush-and-grass mixture fire ...,2016/08/07,04,1315,River,RD,N,,,,,,,0.0,0.0,142,2016/08/07,1,1,True,2016-08-07 12:22:14,2016-08-07 13:00:02,NaN,NaN
38249,NaN,0027623,Passenger vehicle fire ...,2016/08/07,15,,18,AVE,NE,,,NE,Lincoln,ST,,0.0,0.0,131,2016/08/07,1,1,True,2016-08-07 13:32:44,2016-08-07 13:45:29,NaN,NaN
38254,NaN,0027945,Brush or brush-and-grass mixture fire ...,2016/08/10,04,1401,River,RD,N,,,,,,,0.0,0.0,142,2016/08/10,1,1,True,2016-08-10 00:46:49,2016-08-10 01:31:25,NaN,NaN
38256,NaN,0027961,"Outside rubbish, trash or waste fire ...",2016/08/10,08,304,Lake,ST,E,,,,,,,0.0,0.0,151,2016/08/10,1,1,True,2016-08-10 05:31:14,2016-08-10 05:51:53,NaN,NaN
38257,NaN,0028044,Dumpster or other outside trash receptacle fir...,2016/08/10,14,200-201,Interstate 94 Wb,,,,,,,,,0.0,0.0,154,2016/08/10,1,1,True,2016-08-10 17:35:21,2016-08-10 18:14:19,NaN,NaN
38265,NaN,0028438,Passenger vehicle fire ...,2016/08/13,04,525,7,ST,N,,,,,,,0.0,0.0,131,2016/08/13,1,1,True,2016-08-13 10:04:18,2016-08-13 10:33:06,NaN,NaN
38273,NaN,0028946,Passenger vehicle fire ...,2016/08/17,06,100,Interstate 94 Eb,,,,,,,,,0.0,0.0,131,2016/08/17,1,1,True,2016-08-17 09:46:12,2016-08-17 10:47:25,NaN,NaN


In [56]:
fires.loc[fires.new_lat.isna()]

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime,new_lat,new_lon
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14,NaN,NaN
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18,NaN,NaN
38244,NaN,0027398,"Natural vegetation fire, Other ...",2016/08/05,05,,24,AVE,S,,,E,32,ST,,0.0,0.0,140,2016/08/05,1,1,True,2016-08-05 18:35:03,2016-08-05 18:44:37,NaN,NaN
38248,NaN,0027619,Brush or brush-and-grass mixture fire ...,2016/08/07,04,1315,River,RD,N,,,,,,,0.0,0.0,142,2016/08/07,1,1,True,2016-08-07 12:22:14,2016-08-07 13:00:02,NaN,NaN
38249,NaN,0027623,Passenger vehicle fire ...,2016/08/07,15,,18,AVE,NE,,,NE,Lincoln,ST,,0.0,0.0,131,2016/08/07,1,1,True,2016-08-07 13:32:44,2016-08-07 13:45:29,NaN,NaN
38254,NaN,0027945,Brush or brush-and-grass mixture fire ...,2016/08/10,04,1401,River,RD,N,,,,,,,0.0,0.0,142,2016/08/10,1,1,True,2016-08-10 00:46:49,2016-08-10 01:31:25,NaN,NaN
38256,NaN,0027961,"Outside rubbish, trash or waste fire ...",2016/08/10,08,304,Lake,ST,E,,,,,,,0.0,0.0,151,2016/08/10,1,1,True,2016-08-10 05:31:14,2016-08-10 05:51:53,NaN,NaN
38257,NaN,0028044,Dumpster or other outside trash receptacle fir...,2016/08/10,14,200-201,Interstate 94 Wb,,,,,,,,,0.0,0.0,154,2016/08/10,1,1,True,2016-08-10 17:35:21,2016-08-10 18:14:19,NaN,NaN
38265,NaN,0028438,Passenger vehicle fire ...,2016/08/13,04,525,7,ST,N,,,,,,,0.0,0.0,131,2016/08/13,1,1,True,2016-08-13 10:04:18,2016-08-13 10:33:06,NaN,NaN
38273,NaN,0028946,Passenger vehicle fire ...,2016/08/17,06,100,Interstate 94 Eb,,,,,,,,,0.0,0.0,131,2016/08/17,1,1,True,2016-08-17 09:46:12,2016-08-17 10:47:25,NaN,NaN


In [57]:
fires.drop(['latitude', 'longitude'], axis=1).rename(columns={'new_lat':'latitude', 'new_lon':'longitude'}).to_csv('../data/confirmed_fires.csv', index=False)

In [17]:
confirmed_fires = pd.read_csv('../data/confirmed_fires.csv')

In [18]:
cols_to_drop = ['number', 'street', 'st_type', 'st_suffix', 'addr_2', 'apt_room', 'xst_prefix', 'xstreet', 'xst_type', 'xst_suffix']
cols_to_drop.extend(['inci_no', 'inci_type', 'descript', 'alarms', 'confirmed_fire', 'alarm_datetime', 'clear_datetime', 'complete'])
confirmed_fires = confirmed_fires.drop(cols_to_drop, axis=1).rename(columns={'alm_date':'alarm_date', 'clr_date':'clear_date'})

In [19]:
confirmed_fires.head()

,FID,alarm_date,station,clear_date,latitude,longitude
0,NaN,2016/08/02,21,2016/08/02,NaN,NaN
1,NaN,2016/08/02,06,2016/08/02,44.962750,-93.305485
2,NaN,2016/08/02,07,2016/08/02,44.961700,-93.261202
3,NaN,2016/08/03,05,2016/08/03,44.891355,-93.246812
4,NaN,2016/08/03,21,2016/08/03,NaN,NaN


In [20]:
confirmed_fires.to_csv('combined_fires_for_app.csv', index=False)